# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
city_file="output_data/weather.csv"
cities_df = pd.read_csv(city_file)
cities_df

,City,Country,lat,lng,temp,max temp,humidity,cloud,wind speed
0,Yellowknife,CA,62.46,-114.35,22.73,23.00,73,90,9.17
1,Vaini,TO,-21.20,-175.20,71.60,71.60,78,90,5.82
2,Xam Nua,LA,20.42,104.04,70.92,70.92,58,90,4.29
3,Cape Town,ZA,-33.93,18.42,59.50,63.00,67,0,13.87
4,Pindamonhangaba,BR,-22.92,-45.46,61.00,61.00,95,9,0.72
...,...,...,...,...,...,...,...,...,...
548,Tūkrah,LY,32.53,20.58,71.31,71.31,54,0,12.44
549,Minot,US,48.23,-101.30,25.93,26.60,73,90,17.22
550,Turukhansk,RU,65.82,87.98,24.51,24.51,98,100,13.67
551,Guarapari,BR,-20.67,-40.50,68.00,68.00,94,40,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Access maps with API key
gmaps.configure(api_key=g_key)

In [9]:
#Use the Lat and Lng as locations
locations=cities_df[["lat", "lng"]].astype(float)
#set weight
humidity=cities_df["humidity"].astype(float)
#create humidity map
fig = gmaps.figure()

humi_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
fig.add_layer(humi_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
#find ideal cities for living
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
ideal_df=cities_df.loc[(cities_df["max temp"] <= 80)&(cities_df["max temp"]>70)&(cities_df["wind speed"]<10)&(cities_df["cloud"]==0),:]
#drop na
clean_ideal_df=ideal_df.dropna(how='any')
#new dataframe
clean_ideal_df

,City,Country,lat,lng,temp,max temp,humidity,cloud,wind speed
83,Mbamba,NG,9.98,13.10,79.02,79.02,48,0,2.75
90,Iranshahr,IR,27.20,60.68,77.00,77.00,7,0,6.93
178,Abu Samrah,SY,35.30,37.18,78.08,78.08,25,0,3.58
214,Cabo San Lucas,MX,22.89,-109.91,78.01,78.01,69,0,1.01
259,Gusau,NG,12.16,6.66,77.49,77.49,37,0,5.97
266,Urubichá,BO,-15.38,-62.95,70.45,70.45,89,0,1.45
275,Dhārchula,IN,29.85,80.53,77.74,77.74,24,0,6.55
312,Bāfq,IR,31.61,55.41,75.11,75.11,14,0,0.67
344,Sakakah,SA,29.97,40.21,75.20,75.20,13,0,7.52
393,Taoudenni,ML,22.68,-3.98,75.90,75.90,19,0,7.31


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
#create hotel df
hotel_df=clean_ideal_df
#add column
hotel_df["Hotel Name"]=""
hotel_df

<ipython-input-24-6af2fba94ad9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,lat,lng,temp,max temp,humidity,cloud,wind speed,Hotel Name
83,Mbamba,NG,9.98,13.10,79.02,79.02,48,0,2.75,
90,Iranshahr,IR,27.20,60.68,77.00,77.00,7,0,6.93,
178,Abu Samrah,SY,35.30,37.18,78.08,78.08,25,0,3.58,
214,Cabo San Lucas,MX,22.89,-109.91,78.01,78.01,69,0,1.01,
259,Gusau,NG,12.16,6.66,77.49,77.49,37,0,5.97,
266,Urubichá,BO,-15.38,-62.95,70.45,70.45,89,0,1.45,
275,Dhārchula,IN,29.85,80.53,77.74,77.74,24,0,6.55,
312,Bāfq,IR,31.61,55.41,75.11,75.11,14,0,0.67,
344,Sakakah,SA,29.97,40.21,75.20,75.20,13,0,7.52,
393,Taoudenni,ML,22.68,-3.98,75.90,75.90,19,0,7.31,


In [25]:
#para
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}
#use lat & lng to identify location
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["lng"]

    # change location each iteration
    params["location"] = f"{lat},{lng}"

    # url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    # convert to json
    hotel_name = hotel_name.json()
    #skip missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#save to csv
hotel_df.to_csv("output_data/hotel_Output.csv")
#show df
hotel_df

C:\Users\Study\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,lat,lng,temp,max temp,humidity,cloud,wind speed,Hotel Name
83,Mbamba,NG,9.98,13.10,79.02,79.02,48,0,2.75,Hotel
90,Iranshahr,IR,27.20,60.68,77.00,77.00,7,0,6.93,هتل خنونه سرزه
178,Abu Samrah,SY,35.30,37.18,78.08,78.08,25,0,3.58,Resort Safari Souria منتجع سفاري سورية
214,Cabo San Lucas,MX,22.89,-109.91,78.01,78.01,69,0,1.01,Nobu Hotel Los Cabos
259,Gusau,NG,12.16,6.66,77.49,77.49,37,0,5.97,Green Scenery Hotel
266,Urubichá,BO,-15.38,-62.95,70.45,70.45,89,0,1.45,
275,Dhārchula,IN,29.85,80.53,77.74,77.74,24,0,6.55,"Hotel Bala Paradise, Munsiyari"
312,Bāfq,IR,31.61,55.41,75.11,75.11,14,0,0.67,Alamdar Hotel
344,Sakakah,SA,29.97,40.21,75.20,75.20,13,0,7.52,Raoum Inn Hotel
393,Taoudenni,ML,22.68,-3.98,75.90,75.90,19,0,7.31,


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [27]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
#fig
fig.add_layer(markers)
#combine with humidity heatmap
fig = gmaps.figure()

fig.add_layer(humi_layer)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))